# The Price is Right

## Week 8 Order of Play

Day 1: Modal.com and SpecialistAgent  
Day 2: RAG, FrontierAgent, Ensemble Agent  
Day 3: ScannerAgent, MessengerAgent  
Day 4: AutonomousPlannerAgent and DealAgentFramework  
Day 5: The Price Is Right Finale

## RAG (Retrieval Augmented Generation) based on a dataset of 800,000 scraped Amazon products

#### For our 2nd agent, we will be asking OpenAI to estimate the price of one of our deals - and we will give it a hand.

We discovered that LLMs are really good at this, out of the box.

And we discovered that we can beat a frontier LLM by fine-tuning an open-source LLM.

Now we are going to try **inference time** techniques instead of training -- by using RAG!

In [3]:
# imports

import os
import logging
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from litellm import completion
from tqdm.notebook import tqdm
from agents.evaluator import evaluate
from agents.items import Item

In [4]:
# environment

load_dotenv(override=True)
DB = "products_vectorstore"

In [5]:
# Log in to HuggingFace
# If you don't have a HuggingFace account, you can set one up for free at www.huggingface.co
# And then add the HF_TOKEN to your .env file as explained in the project README

hf_token = os.environ['HF_TOKEN']
login(token=hf_token, add_to_git_credential=False)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [6]:
LITE_MODE = True

In [7]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


# Now create a Chroma Datastore

Now we will use the free, open-source Vector database Chroma.  
We will create a Chroma datastore with 400,000 products from our training dataset.

In [8]:
client = chromadb.PersistentClient(path=DB)
print(DB)

products_vectorstore


# Introducing the SentenceTransformer Encoding LLM

The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to 384 dimensional vectors and is ideal for tasks like semantic search.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

It can run pretty quickly locally.

As an alternative, OpenAI provides a closed-source Embeddings model. Benefits compared to OpenAI embeddings:
1. It's free and fast!
3. We can run it locally, so the data never leaves our box - might be useful if you're building a personal RAG

In [8]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [9]:
# Pass in a list of texts, get back a numpy array of vectors

vector = encoder.encode(["A proficient AI engineer who has almost reached the finale of AI Engineering Core Track!"])[0]
print(vector.shape)
vector

(384,)


array([-5.68234064e-02, -6.70466274e-02,  4.41129990e-02,  5.98602742e-03,
       -2.28948686e-02, -2.95300409e-02,  5.56368940e-02,  3.42665277e-02,
       -1.08529389e-01, -3.81690413e-02, -7.43872151e-02, -1.03664294e-01,
        1.69148333e-02,  1.33925094e-03, -6.86191097e-02,  8.99353474e-02,
       -1.45185888e-02, -2.43885349e-02,  4.21858160e-03, -9.62992534e-02,
       -2.51799300e-02,  4.60676402e-02,  4.95346216e-03, -3.88679653e-02,
        1.07714802e-03,  6.82337433e-02, -1.13859829e-02, -5.83416224e-02,
       -1.03801349e-02, -1.74953304e-02, -1.86478384e-02,  4.07060608e-03,
        1.59438029e-02,  6.49721697e-02,  3.71175818e-02,  2.78224889e-02,
       -4.41945232e-02, -2.34372616e-02,  9.71035808e-02, -5.06138988e-02,
       -1.93864331e-02, -3.83471511e-02,  4.76067066e-02, -3.36107239e-02,
        5.08287847e-02,  3.57934721e-02,  2.91818311e-03, -1.06529139e-01,
        4.07211818e-02, -5.85388450e-04, -1.05607413e-01, -1.03584379e-01,
        3.71124037e-02, -

## With that background, let's populate our Chroma database

### By calculating vectors for 800,000 scraped products

This takes 30 minutes on my machine on my GPU - it might take longer for you - feel free to use the Lite dataset!

In [9]:
# Check if the collection exists; if not, create it

collection_name = "products"
existing_collection_names = [collection.name for collection in client.list_collections()]

if collection_name not in existing_collection_names:
    collection = client.create_collection(collection_name)
    for i in tqdm(range(0, len(train), 1000)):
        documents = [item.summary for item in train[i: i+1000]]
        vectors = encoder.encode(documents).astype(float).tolist()
        metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
        ids = [f"doc_{j}" for j in range(i, i+1000)]
        ids = ids[:len(documents)]
        collection.add(ids=ids, documents=documents, embeddings=vectors, metadatas=metadatas)

collection = client.get_or_create_collection(collection_name)

# Let's visualize the vectorized data

In [11]:
# It is very fun turning this up to 800_000 and seeing the full dataset visualized,
# but it almost crashes my box every time so do that at your own risk!! 10_000 is safe!

MAXIMUM_DATAPOINTS = 10_000

In [15]:
CATEGORIES = ['Appliances', 'Automotive', 'Cell_Phones_and_Accessories', 'Electronics','Musical_Instruments', 'Office_Products', 'Tools_and_Home_Improvement', 'Toys_and_Games']
COLORS = ['cyan', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'red']

In [13]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'], limit=MAXIMUM_DATAPOINTS)
vectors = np.array(result['embeddings'])
documents = result['documents']
categories = [metadata['category'] for metadata in result['metadatas']]
colors = [COLORS[CATEGORIES.index(c)] for c in categories]

In [16]:
# Let's try a 2D chart
# TSNE stands for t-distributed Stochastic Neighbor Embedding - it's a common technique for reducing dimensionality of data

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [ ]:
# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=4, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vectorstore Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [ ]:
# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=2, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [31]:
test[0]

<Old Blood Noise Excess V2 Distortion Chorus/Delay Pedal = $219.0>

In [32]:
def vector(item):
    return encoder.encode(item.summary)

In [33]:
def find_similars(item):
    vec = vector(item)
    results = collection.query(query_embeddings=vec.astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [34]:
find_similars(test[0])

(['Title: Digitech DOD Carcosa Fuzz Distortion Pedal  \nCategory: Electronics  \nBrand: DigiTech  \nDescription: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  \nDetails: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑mid boost, and a 9‑V power supply.',
  'Title: EarthQuaker Devices Plumes Small Signal Shredder – Purple Sparkle  \nCategory: Guitar Pedals  \nBrand: EarthQuaker Devices  \nDescription: An all‑analog overdrive pedal that delivers three distinct clipping voices and expansive headroom for crystal‑clear, three‑dimensional tone.  \nDetails: Features Flexi‑Switch relay‑based bypass, 9\u202fV power input, and a finely tuned tone control for sculpting low, mid, and high frequencies.',
  'Title: Dunlop Rotovibe Pedal  \nCategory: Audio Effects  \nBrand: Dunlop  \nDescription: A versatile chorus/vibrato pedal that simulates rotating speaker sounds w

In [20]:
# We need to give some context to GPT-5.1 by selecting 5 products with similar descriptions

def make_context(similars, prices):
    message = "For context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [28]:
documents, prices = find_similars(test[0])
print(make_context(documents, prices))

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Digitech DOD Carcosa Fuzz Distortion Pedal  
Category: Electronics  
Brand: DigiTech  
Description: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  
Details: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑mid boost, and a 9‑V power supply.
Price is $159.99

Potentially related product:
Title: EarthQuaker Devices Plumes Small Signal Shredder – Purple Sparkle  
Category: Guitar Pedals  
Brand: EarthQuaker Devices  
Description: An all‑analog overdrive pedal that delivers three distinct clipping voices and expansive headroom for crystal‑clear, three‑dimensional tone.  
Details: Features Flexi‑Switch relay‑based bypass, 9 V power input, and a finely tuned tone control for sculpting low, mid, and high frequencies.
Price is $99.00


In [22]:
def messages_for(item, similars, prices):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}\n\n"
    message += make_context(similars, prices)
    return [{"role": "user", "content": message}]

In [23]:
documents, prices = find_similars(test[0])
print(messages_for(test[0], documents, prices)[0]['content'])

Estimate the price of this product. Respond with the price, no explanation

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Digitech DOD Carcosa Fuzz Distortion Pedal  
Category: Electronics  
Brand: DigiTech  
Description: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  
Details: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑

In [24]:
# The function for gpt-5-mini

def gpt_5__1_rag(item):
    documents, prices = find_similars(item)
    response = completion(model="gpt-5.1", messages=messages_for(item, documents, prices), reasoning_effort="none", seed=42)
    return response.choices[0].message.content

In [25]:
# How much does our favorite distortion pedal cost?

test[0].price

219.0

In [26]:
# Let's do this!!

gpt_5__1_rag(test[0])

'$229.00'

In [27]:
evaluate(gpt_5__1_rag, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$11 $54 $5 $20 $20 $30 $24 $80 $4 $220 $63 $20 $0 $9 $29 $5 $11 $17 $50 $29 $41 $6 $40 $15 $82 $264 $65 $5 $111 $62 $35 $25 $40 $32 $20 $30 $37 $31 $24 $13 $155 $15 $16 $75 $80 $8 $32 $4 $75 $128 $24 $87 $130 $5 $27 $76 $9 $40 $123 $13 $86 $45 $41 $30 $20 $0 $20 $216 $5 $124 $18 $2 $40 $1 $10 $17 $4 $1 $3 $7 $0 $3 $11 $16 $13 $70 $78 $141 $0 $6 $3 $5 $1 $1 $0 $108 $2 $37 $90 $125 $10 $23 $13 $20 $0 $71 $15 $325 $12 $80 $25 $36 $9 $8 $54 $30 $10 $1 $134 $117 $9 $9 $20 $36 $19 $10 $1 $71 $19 $59 $0 $47 $5 $5 $35 $1 $15 $79 $13 $41 $4 $49 $20 $4 $4 $8 $5 $161 $5 $8 $0 $54 $25 $21 $21 $11 $41 $31 $10 $10 $10 $14 $5 $5 $320 $0 $181 $22 $10 $4 $11 $3 $170 $5 $27 $35 $3 $17 $26 $7 $75 $15 $171 $36 $17 $18 $53 $27 $10 $8 $5 $1 $5 $11 $0 $10 $19 $2 $18 $7 

In [35]:
import modal
Pricer = modal.Cls.from_name("pricer-service", "Pricer")
pricer = Pricer()

In [36]:
def specialist(item):
    return pricer.price.remote(item.summary)


In [37]:
def get_price(reply):
    reply = reply.replace("$", "").replace(",", "")
    match = re.search(r"[-+]?\d*\.\d+|\d+", reply)
    return float(match.group()) if match else 0

## Download the Neural Network weights from Week 6 into this directory

The file `deep_neural_network.pth` here:

https://drive.google.com/drive/folders/1uq5C9edPIZ1973dArZiEO-VE13F7m8MK?usp=drive_link

In [ ]:

from agents.deep_neural_network import DeepNeuralNetworkInference

runner = DeepNeuralNetworkInference()
runner.setup()
runner.load("deep_neural_network.pth")

def deep_neural_network(item):
    return runner.inference(item.summary)

In [ ]:
def ensemble(item):
    price1 = get_price(gpt_5__1_rag(item))
    price2 = specialist(item)
    price3 = deep_neural_network(item)
    return price1 * 0.8 + price2 * 0.1 + price3 * 0.1


In [ ]:
evaluate(ensemble, test)

In [42]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [43]:
from agents.frontier_agent import FrontierAgent

agent = FrontierAgent(collection)
agent.price("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")

INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $119.99


119.99

In [44]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $279.00


279.0

In [ ]:
from agents.neural_network_agent import NeuralNetworkAgent
agent = NeuralNetworkAgent()


In [ ]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

In [10]:
from agents.ensemble_agent import EnsembleAgent
agent = EnsembleAgent(collection)

In [11]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

283.0